### Run this in Python kernel

To reset font style of output (in case it is not the default one)

In [1]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            ;;[clojure-backtesting.parameters :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojure.edn :as edn]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset

In [2]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative path to your own dataset
;
(reset! data-set (add-aprc (read-csv-row "../resources/CRSP-extract.csv")));

### Get list of tickers with ROE

In [3]:
;; Use merged database CRSP + COMPUSTAT
;; Compute the latest quarter ROE by using: niq/(share price * cshoq) in the merged dataset
;; output {{:tic "AAPL" :year "1980" :ROE x.x}{...}{:ROE 1.x 2.x 3.x 10.x}}

;; need to use lazy sequence if applying on big dataset

(defn get-set-roe
    "return a set of maps of tickers and datadate" ;;{:tic "AAPL", :datadate "1981/3/31", :ROE x.x}
    [file date]
    (loop [remaining file
            result-set []
            ROE-list [] ]
        (if (empty? remaining)
            ;; how to change the output here?
            ;;(into #{} (conj result-set {:ROE-set ROE-list}))
            {:data result-set :ROE-set ROE-list}
            (let [first-line (first remaining)
                next-remaining (rest remaining)
                ;;next-result-set (conj result-set (get first-line :datadate)
                [year month day] (map parse-int (str/split date #"-"))]
                ;;merged file use :datadate as key
                (if (= date (get first-line :datadate))
                    (let [[niq PRC cshoq] (map edn/read-string [(get first-line :niq)(get first-line :PRC)(get first-line :cshoq)])
                          ROE (/ niq (* PRC cshoq))]
                        (recur next-remaining (conj result-set {:tic (get first-line :tic) :year year :ROE ROE}) (conj ROE-list ROE))
                    )
                    (recur next-remaining result-set ROE-list)
                )  
            )
        )
    )
)

#'clojure-backtesting.demo/get-set-roe

In [4]:
;; demo
(def dataFA (get-set-roe (read-csv-row "../resources/data-testing-merged.csv") "1980-12-18"))
(println dataFA) ;

{:data [{:tic AAPL, :year 1980, :ROE 0.002731506525434953} {:tic GM, :year 1980, :ROE -0.046370846428037134} {:tic IBM, :year 1980, :ROE 0.0238072629827958}], :ROE-set [0.002731506525434953 -0.046370846428037134 0.0238072629827958]}


In [5]:
;; fn to get a set of ROE
(defn get-ROE
            "return a set of ROE"  ;;{10.2 1.8 x.x ...}
        [dataset]
        (into #{} (get dataset :ROE-set))
        )

#'clojure-backtesting.demo/get-ROE

In [6]:
(defn get-roe-20
    "return a set tickers" ;;{"AAPL" "GM"}
    [data roe-20]
    (loop [remaining (get data :data)
            result-set []]
        (if (empty? remaining)
            (into #{} result-set)
            (let [first-line (first remaining)
                next-remaining (rest remaining)]
                (if (>= (get first-line :ROE) roe-20)
                    (recur next-remaining (conj result-set (get first-line :tic)))           
                    (recur next-remaining result-set)            
                )  
            )
        )
    )
)

#'clojure-backtesting.demo/get-roe-20

In [7]:
;; test examples

;; dataFA is the dataset that contains the output from get-set

;; get a set of ROE
(def roe-list-demo (get-ROE dataFA))
(println roe-list-demo)


;; get a sorted set of ROE
(def roe-sorted-set (apply sorted-set roe-list-demo))
(println roe-sorted-set)


;; get a sorted vector of ROE
(def roe-sorted-vec (into '[] roe-sorted-set))
(println roe-sorted-vec)

;; find the 20th per centile ROE
(def roe-20-demo (nth roe-sorted-vec (int (* 0.8 (count roe-sorted-vec)))))
(println roe-20-demo)
         
;; get the list of tickers to buy
(def stock-to-buy-test (get-roe-20 dataFA roe-20-demo))
(println stock-to-buy-test)
(println (type stock-to-buy-test)) ;;stored in a set format

#{0.0238072629827958 -0.046370846428037134 0.002731506525434953}
#{-0.046370846428037134 0.002731506525434953 0.0238072629827958}
[-0.046370846428037134 0.002731506525434953 0.0238072629827958]
0.0238072629827958
#{IBM}
clojure.lang.PersistentHashSet


nil

### Initialise portfolio

In [8]:
(init-portfolio "1980-12-18" 100000)

#'clojure-backtesting.order/portfolio-value

In [9]:
(keys (deref available-tics))

("AAPL" "F" "IBM")

In [10]:
(def year-count 3) ;; hold the stock for 3 years
(def start-year 1980)
(def rebalance-md (subs (get-date) 4)) ; = -12-18

(def rebalance-years (into [] (range (+ start-year 1) (+ (+ start-year 1) year-count) 1))) ; rebalance every year

(def rebalance-dates []) ; [1981-12-18, 1982-12-18, 1983-12-18]
(doseq [year rebalance-years]
  (def rebalance-dates (conj rebalance-dates (str year rebalance-md)))
)

(def end-date (last rebalance-dates)) ; 1983-12-18

;; get stock tickers and ROE data
;; output {{:tic "AAPL" :year "1980" :ROE x.x}{...}{:ROE 1.x 2.x 3.x 10.x}}
(def stock-data (get-set-roe (read-csv-row "../resources/data-testing-merged.csv") "1980-12-18"))

;; sort the stocks according to their ROE (= Net Income/Total Equity)

;; get a set of ROE
(def roe-list (get-ROE stock-data))

;; sorting function, return tickers of the top 20% stocks
;; determine what are the top 20% stocks and turn the sorted-set into list format
(def roe-sorted (into '[] (apply sorted-set roe-list)))

;(println roe-list)
;(println (int (* 0.8 (count roe-sorted))))

;; find the 20% cut-off ROE value
(let [roe-20 (nth roe-sorted (int (* 0.8 (count roe-sorted))))]    
    ;;get the tickers of the top 20 with function get-roe-20
    (def stocks-to-buy (get-roe-20 stock-data roe-20))
)

;; buy the top 20% stocks and the sell the stocks that are not in the top 20% this year      
(def stocks-to-buy-list (into [] stocks-to-buy))

;; buy stocks 
(doseq [stock stocks-to-buy-list]        
  (order stock 10)
)

(update-eval-report (get-date)) ; update evaluation metrics
(next-date)

(while (not= (empty? rebalance-dates) true)
  ;(println (get-date)) ; debug
  (if (t/after? (t/local-date (get-date)) (t/local-date (first rebalance-dates))) ; check if (get-date) has passed first date in rebalance-dates
    (do
      (def rebalance-dates (rest rebalance-dates)) ; pop the first date in rebalance-dates
      ;(println (rest rebalance-dates)) ; debug
      (while (empty? (get-set-roe (read-csv-row "../resources/data-testing-merged.csv") (get-date)))
          (next-date) ;; move on to next date til data is not empty
      )
      (def stock-data (get-set-roe (read-csv-row "../resources/data-testing-merged.csv") (get-date)))
      (def roe-list (get-ROE stock-data))
      (def roe-sorted (into '[] (apply sorted-set roe-list)))
      (let [roe-20 (nth roe-sorted (int (* 0.8 (count roe-sorted))))]    
          (def stocks-to-buy (get-roe-20 stock-data roe-20))
      )
      (def stocks-to-buy-list (into [] stocks-to-buy))

      ;; sell stocks held in portfolio
      (doseq [[ticker row] (deref portfolio)]
        (if (not= ticker :cash)      
          (order ticker -10)
        )
      )

      ;; buy stocks
      (doseq [stock stocks-to-buy-list]       
        (order stock 10)
      )

      (update-eval-report (get-date)) ; update evaluation metrics
    )
  )
  (next-date) ; move on to the next trading day
)

;; sell stocks held in portfolio (if ticker != "cash" && quantity > 0)
(doseq [[ticker row] (deref portfolio)]
  (if (and (not= ticker :cash) (= (compare (get row :quantity) 0) 1))
      (order ticker -10)
  )
)

The order request 1982-01-08 | IBM | 0 fails.
Failure reason: No such date or ticker in the dataset or the dataset has reached the end.
The order request 1983-12-19 | GM | 10 fails.
Failure reason: No such date or ticker in the dataset or the dataset has reached the end.


nil

### Check order record

In [11]:
(pprint/print-table (deref order-record))


|      :date | :tic | :price |   :aprc | :quantity |
|------------+------+--------+---------+-----------|
| 1980-12-19 |  IBM | 64.625 |  736.07 |        10 |
| 1981-12-22 |  IBM | 56.875 |  714.58 |       -10 |
| 1981-12-22 |  IBM | 56.875 |  714.58 |        10 |
| 1982-12-21 |  IBM |  95.75 |  915.74 |       -10 |
| 1982-12-21 |  IBM |  95.75 |  915.74 |        10 |
| 1983-12-20 |  IBM | 121.75 | 1030.91 |       -10 |


nil

### Check portfolio record

In [13]:
;; view final portfolio
(view-portfolio)


| :asset | :price |   :aprc | :quantity | :tot-val |
|--------+--------+---------+-----------+----------|
|   cash |    N/A |     N/A |       N/A |   100571 |
|    IBM | 121.75 | 1030.91 |         0 |        0 |


nil

In [14]:
;; view portfolio value and return
(view-portfolio-record)


|      :date | :tot-value | :daily-ret | :tot-ret | :loan | :leverage |
|------------+------------+------------+----------+-------+-----------|
| 1980-12-18 |    $100000 |      0.00% |    0.00% | $0.00 |     0.00% |
| 1980-12-19 |    $106714 |      0.00% |    6.50% | $0.00 |     0.00% |
| 1980-12-22 |    $106836 |      0.11% |    6.61% | $0.00 |     0.00% |
| 1980-12-23 |    $106842 |      0.01% |    6.62% | $0.00 |     0.00% |
| 1980-12-24 |    $106903 |      0.06% |    6.68% | $0.00 |     0.00% |
| 1980-12-26 |    $106926 |      0.02% |    6.70% | $0.00 |     0.00% |
| 1980-12-29 |    $106879 |     -0.04% |    6.65% | $0.00 |     0.00% |
| 1980-12-30 |    $106908 |      0.03% |    6.68% | $0.00 |     0.00% |
| 1980-12-31 |    $106873 |     -0.03% |    6.65% | $0.00 |     0.00% |
| 1981-01-02 |    $106932 |      0.06% |    6.70% | $0.00 |     0.00% |
| 1981-01-05 |    $106986 |      0.05% |    6.75% | $0.00 |     0.00% |
| 1981-01-06 |    $107044 |      0.05% |    6.81% | $0.00 |    

| 1981-06-01 |    $106581 |      0.07% |    6.37% | $0.00 |     0.00% |
| 1981-06-02 |    $106562 |     -0.02% |    6.36% | $0.00 |     0.00% |
| 1981-06-03 |    $106522 |     -0.04% |    6.32% | $0.00 |     0.00% |
| 1981-06-04 |    $106463 |     -0.06% |    6.26% | $0.00 |     0.00% |
| 1981-06-05 |    $106483 |      0.02% |    6.28% | $0.00 |     0.00% |
| 1981-06-08 |    $106476 |     -0.01% |    6.28% | $0.00 |     0.00% |
| 1981-06-09 |    $106429 |     -0.04% |    6.23% | $0.00 |     0.00% |
| 1981-06-10 |    $106450 |      0.02% |    6.25% | $0.00 |     0.00% |
| 1981-06-11 |    $106516 |      0.06% |    6.31% | $0.00 |     0.00% |
| 1981-06-12 |    $106536 |      0.02% |    6.33% | $0.00 |     0.00% |
| 1981-06-15 |    $106562 |      0.02% |    6.36% | $0.00 |     0.00% |
| 1981-06-16 |    $106509 |     -0.05% |    6.31% | $0.00 |     0.00% |
| 1981-06-17 |    $106489 |     -0.02% |    6.29% | $0.00 |     0.00% |
| 1981-06-18 |    $106443 |     -0.04% |    6.24% | $0.00 |     

| 1981-11-10 |    $106241 |      0.03% |    6.05% | $0.00 |     0.00% |
| 1981-11-11 |    $106255 |      0.01% |    6.07% | $0.00 |     0.00% |
| 1981-11-12 |    $106269 |      0.01% |    6.08% | $0.00 |     0.00% |
| 1981-11-13 |    $106205 |     -0.06% |    6.02% | $0.00 |     0.00% |
| 1981-11-16 |    $106169 |     -0.03% |    5.99% | $0.00 |     0.00% |
| 1981-11-17 |    $106169 |      0.00% |    5.99% | $0.00 |     0.00% |
| 1981-11-18 |    $106132 |     -0.03% |    5.95% | $0.00 |     0.00% |
| 1981-11-19 |    $106132 |      0.00% |    5.95% | $0.00 |     0.00% |
| 1981-11-20 |    $106147 |      0.01% |    5.97% | $0.00 |     0.00% |
| 1981-11-23 |    $106140 |     -0.01% |    5.96% | $0.00 |     0.00% |
| 1981-11-24 |    $106241 |      0.10% |    6.05% | $0.00 |     0.00% |
| 1981-11-25 |    $106262 |      0.02% |    6.07% | $0.00 |     0.00% |
| 1981-11-27 |    $106368 |      0.10% |    6.17% | $0.00 |     0.00% |
| 1981-11-30 |    $106368 |      0.00% |    6.17% | $0.00 |     

| 1982-04-27 |    $106977 |     -0.04% |    6.74% | $0.00 |     0.00% |
| 1982-04-28 |    $106964 |     -0.01% |    6.73% | $0.00 |     0.00% |
| 1982-04-29 |    $106926 |     -0.04% |    6.70% | $0.00 |     0.00% |
| 1982-04-30 |    $106932 |      0.01% |    6.70% | $0.00 |     0.00% |
| 1982-05-03 |    $106945 |      0.01% |    6.71% | $0.00 |     0.00% |
| 1982-05-04 |    $106958 |      0.01% |    6.73% | $0.00 |     0.00% |
| 1982-05-05 |    $106977 |      0.02% |    6.74% | $0.00 |     0.00% |
| 1982-05-06 |    $107008 |      0.03% |    6.77% | $0.00 |     0.00% |
| 1982-05-07 |    $107001 |     -0.01% |    6.77% | $0.00 |     0.00% |
| 1982-05-10 |    $106956 |     -0.04% |    6.73% | $0.00 |     0.00% |
| 1982-05-11 |    $106982 |      0.02% |    6.75% | $0.00 |     0.00% |
| 1982-05-12 |    $106956 |     -0.02% |    6.73% | $0.00 |     0.00% |
| 1982-05-13 |    $106943 |     -0.01% |    6.71% | $0.00 |     0.00% |
| 1982-05-14 |    $106943 |      0.00% |    6.71% | $0.00 |     

| 1982-10-07 |    $107803 |      0.06% |    7.51% | $0.00 |     0.00% |
| 1982-10-08 |    $107837 |      0.03% |    7.55% | $0.00 |     0.00% |
| 1982-10-11 |    $107944 |      0.10% |    7.64% | $0.00 |     0.00% |
| 1982-10-12 |    $107910 |     -0.03% |    7.61% | $0.00 |     0.00% |
| 1982-10-13 |    $107944 |      0.03% |    7.64% | $0.00 |     0.00% |
| 1982-10-14 |    $107791 |     -0.14% |    7.50% | $0.00 |     0.00% |
| 1982-10-15 |    $107763 |     -0.03% |    7.48% | $0.00 |     0.00% |
| 1982-10-18 |    $107922 |      0.15% |    7.62% | $0.00 |     0.00% |
| 1982-10-19 |    $107882 |     -0.04% |    7.59% | $0.00 |     0.00% |
| 1982-10-20 |    $107989 |      0.10% |    7.69% | $0.00 |     0.00% |
| 1982-10-21 |    $107955 |     -0.03% |    7.66% | $0.00 |     0.00% |
| 1982-10-22 |    $107910 |     -0.04% |    7.61% | $0.00 |     0.00% |
| 1982-10-25 |    $107786 |     -0.12% |    7.50% | $0.00 |     0.00% |
| 1982-10-26 |    $107877 |      0.08% |    7.58% | $0.00 |     

| 1983-03-21 |    $108752 |      0.03% |    8.39% | $0.00 |     0.00% |
| 1983-03-22 |    $108727 |     -0.02% |    8.37% | $0.00 |     0.00% |
| 1983-03-23 |    $108807 |      0.07% |    8.44% | $0.00 |     0.00% |
| 1983-03-24 |    $108812 |      0.00% |    8.45% | $0.00 |     0.00% |
| 1983-03-25 |    $108807 |     -0.00% |    8.44% | $0.00 |     0.00% |
| 1983-03-28 |    $108817 |      0.01% |    8.45% | $0.00 |     0.00% |
| 1983-03-29 |    $108837 |      0.02% |    8.47% | $0.00 |     0.00% |
| 1983-03-30 |    $108892 |      0.05% |    8.52% | $0.00 |     0.00% |
| 1983-03-31 |    $108792 |     -0.09% |    8.43% | $0.00 |     0.00% |
| 1983-04-04 |    $108817 |      0.02% |    8.45% | $0.00 |     0.00% |
| 1983-04-05 |    $108802 |     -0.01% |    8.44% | $0.00 |     0.00% |
| 1983-04-06 |    $108827 |      0.02% |    8.46% | $0.00 |     0.00% |
| 1983-04-07 |    $108852 |      0.02% |    8.48% | $0.00 |     0.00% |
| 1983-04-08 |    $108877 |      0.02% |    8.51% | $0.00 |     

| 1983-08-31 |    $109546 |      0.07% |    9.12% | $0.00 |     0.00% |
| 1983-09-01 |    $109513 |     -0.03% |    9.09% | $0.00 |     0.00% |
| 1983-09-02 |    $109555 |      0.04% |    9.13% | $0.00 |     0.00% |
| 1983-09-06 |    $109633 |      0.07% |    9.20% | $0.00 |     0.00% |
| 1983-09-07 |    $109652 |      0.02% |    9.21% | $0.00 |     0.00% |
| 1983-09-08 |    $109647 |     -0.00% |    9.21% | $0.00 |     0.00% |
| 1983-09-09 |    $109647 |      0.00% |    9.21% | $0.00 |     0.00% |
| 1983-09-12 |    $109620 |     -0.03% |    9.19% | $0.00 |     0.00% |
| 1983-09-13 |    $109638 |      0.02% |    9.20% | $0.00 |     0.00% |
| 1983-09-14 |    $109670 |      0.03% |    9.23% | $0.00 |     0.00% |
| 1983-09-15 |    $109629 |     -0.04% |    9.19% | $0.00 |     0.00% |
| 1983-09-16 |    $109679 |      0.05% |    9.24% | $0.00 |     0.00% |
| 1983-09-19 |    $109702 |      0.02% |    9.26% | $0.00 |     0.00% |
| 1983-09-20 |    $109706 |      0.00% |    9.26% | $0.00 |     

nil

### Generate evaluation report

In [15]:
(eval-report)


|      :date | :pnl-pt | :sharpe | :tot-val |  :vol |
|------------+---------+---------+----------+-------|
| 1980-12-18 |   $6714 |   1.41% |  $106714 | 4.60% |
| 1981-12-21 |   $2166 |  15.72% |  $106499 | 0.40% |
| 1982-12-20 |   $1702 |  23.22% |  $108511 | 0.35% |
| 1983-12-19 |     $95 |   1.80% |  $100571 | 0.32% |


nil

### Plot variables
Below are example codes that show how to plot different variables in the portfolio record / evaluation record.

### 1. Portfolio total value

In [16]:
(def data (deref portfolio-value))

#'clojure-backtesting.demo/data

In [17]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "total value")
  data))

#'clojure-backtesting.demo/data-to-plot

In [18]:
(first data-to-plot)

{:date "1980-12-18", :tot-value 100000, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0, :plot "total value"}

In [19]:
(plot data-to-plot :plot :date :tot-value false)

### 2. Plot sharpe ratio

In [20]:
(def data (deref eval-record))

; Add rename legend
(def data-to-plot
 (map #(assoc % :plot "sharpe ratio")
  data))

(plot data-to-plot :plot :date :sharpe false)

### 3. Plot stock price

In [21]:
(def data (deref order-record))

#'clojure-backtesting.demo/data

In [22]:
(first data)

{:date "1980-12-19", :tic "IBM", :price 64.625, :aprc "736.07", :quantity 10}

In [26]:
; Add rename legend
(def data-to-plot
 (map #(assoc % :plot "price")
  data))

(plot data-to-plot :plot :date :price false)